In [1]:
# from extremeweatherbench import events
from extremeweatherbench import config
from extremeweatherbench import events
from extremeweatherbench import evaluate

import importlib
importlib.reload(config)
import xarray as xr

In [2]:
event_list = [events.HeatWave,
              events.Freeze]
heatwave_configuration = config.Config(
    event_types=event_list,
    )

default_forecast_config = config.ForecastSchemaConfig()


In [3]:
heatwave_configuration

Config(output_dir='assets/data/outputs/', forecast_dir='assets/data/forecasts/', gridded_obs_path='gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3', point_obs_path=None, event_types=[<class 'extremeweatherbench.events.HeatWave'>, <class 'extremeweatherbench.events.Freeze'>], cache=False)

In [18]:
cases = evaluate.evaluate(eval_config=heatwave_configuration, forecast_schema_config=default_forecast_config, dry_run=True)

In [22]:
cases.metrics

[src.ExtremeWeatherBench.metrics.MaximumMAE,
 src.ExtremeWeatherBench.metrics.DurationME,
 src.ExtremeWeatherBench.metrics.RegionalRMSE,
 src.ExtremeWeatherBench.metrics.MaxMinMAE,
 src.ExtremeWeatherBench.metrics.OnsetME]